In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import time
# import seaborn as sns
from xgboost import plot_importance

In [2]:
def num_null(df):
    missing = df.isnull().sum()
    print('Show #missing in the columns:')
    for i in range(df.shape[1]):
        if missing[i]:
            print(missing.index[i], ':', missing[i])


def metric(truth, pred):
    truth = np.array(truth)
    pred = np.array(pred)
    diff = abs(pred - truth) / truth
    print(list(diff <= 0.1).count(True) / len(diff))

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
train['town'] = train['city'] * 1000 + train['town']
test['town'] = test['city'] * 1000 + test['town']

In [5]:
train['total_price'] = np.log1p((train["total_price"] / train["building_area"])) 

In [6]:
X = pd.concat([train.drop(['total_price'], axis=1), test])
X.reset_index(inplace=True, drop=True)

In [7]:
num_null(X)

Show #missing in the columns:
parking_area : 66397
parking_price : 53775
txn_floor : 18541
village_income_median : 1326


In [8]:
pd.options.mode.chained_assignment = None

In [9]:
X_imp = X.copy()
X_imp['parking_area'].fillna(0, inplace=True)
X_imp['parking_price'].fillna(X_imp['parking_price'].median(), inplace=True)
X_imp['txn_floor'].fillna(-1, inplace=True)

In [10]:
vimm = X_imp.groupby('village').agg({'village_income_median':'mean'})

vim_isnan = np.isnan(X_imp['village_income_median'])
for i in range(len(X_imp)):
    if vim_isnan[i]:
        X_imp['village_income_median'][i] = vimm.loc[X_imp['village'][i], 'village_income_median']

/home/rex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
num_null(X_imp)

Show #missing in the columns:
village_income_median : 938


In [13]:
tvimm = X_imp.groupby('town').agg({'village_income_median':'median'})

vim_isnan = np.isnan(X_imp['village_income_median'])
for i in range(len(X_imp)):
    if vim_isnan[i]:
        X_imp['village_income_median'][i] = tvimm.loc[X_imp['town'][i], 'village_income_median']

# knn_impute(X_imp, 'village_income_median', ['lat', 'lon'])

/home/rex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
num_null(X_imp)

Show #missing in the columns:


In [15]:
X_imp.drop(['parking_area','parking_price'], axis=1, inplace=True)

In [16]:
concat_train = train.copy()

In [17]:
building_material_dummies = pd.get_dummies(X_imp['building_material'])
building_material_dummies.columns = ['building_material_' + str(col) for col in building_material_dummies.columns]
X_imp = pd.concat([X_imp, building_material_dummies], axis=1)
X_imp.drop(['building_material', building_material_dummies.columns[0]], inplace=True, axis=1)

city_dummies = pd.get_dummies(X_imp['city'])
city_dummies.columns = ['city_' + str(col) for col in city_dummies.columns]
X_imp = pd.concat([X_imp, city_dummies], axis=1)
X_imp.drop(['city', city_dummies.columns[0]], inplace=True, axis=1)

In [18]:
building_type_dummies = pd.get_dummies(X_imp['building_type'])
building_type_dummies.columns = ['building_type_' + str(col) for col in building_type_dummies.columns]
X_imp = pd.concat([X_imp, building_type_dummies], axis=1)
X_imp.drop(['building_type', building_type_dummies.columns[0]], inplace=True, axis=1)

building_use_dummies = pd.get_dummies(X_imp['building_use'])
building_use_dummies.columns = ['building_use_' + str(col) for col in building_use_dummies.columns]
X_imp = pd.concat([X_imp, building_use_dummies], axis=1)
X_imp.drop(['building_use', building_use_dummies.columns[0]], inplace=True, axis=1)

parking_way_dummies = pd.get_dummies(X_imp['parking_way'])
parking_way_dummies.columns = ['parking_way_' + str(col) for col in parking_way_dummies.columns]
X_imp = pd.concat([X_imp, parking_way_dummies], axis=1)
X_imp.drop(['parking_way', parking_way_dummies.columns[0]], inplace=True, axis=1)

In [20]:
X_imp["village"].head()

0    3132
1     921
2    1544
3    3350
4      63
Name: village, dtype: int64

In [21]:
X_imp.drop(['building_material_5'], axis=1, inplace = True)

In [22]:
post_train = X_imp.head(60000)
X_test = X_imp.tail(10000)

post_train.drop('building_id', axis=1, inplace=True)
post_train['total_price'] = train.head(60000)['total_price']

test_building_id = X_test['building_id']
X_test.drop('building_id', axis=1, inplace=True)

/home/rex/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/rex/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(
                        post_train.drop('total_price', axis=1), post_train['total_price'], test_size=0.2, random_state=42)

In [24]:
model = lgb.LGBMRegressor(learning_rate=0.1, num_leaves=1291, max_bin=500, nthread=20)

ts = time.time()
model.fit(
    X_train, 
    Y_train)
print(time.time() - ts)

50.509499311447144


In [25]:
ts = time.time()

model = lgb.LGBMRegressor(learning_rate=0.1, num_leaves=1291, max_bin=500)

model.fit(
    post_train.drop('total_price', axis=1), 
    post_train['total_price'])

print(time.time() - ts)

41.37378931045532


In [26]:
Y_test_predict = model.predict(X_test)
Y_test_predict = (np.floor(np.expm1(Y_test_predict)) * X_test['building_area'])

In [24]:
submission = pd.DataFrame({
    "building_id": test_building_id, 
    "total_price": Y_test_predict
})
submission.to_csv('test.csv', index=False)